In [19]:
import torch
from torchvision.models import resnet18
from torchinfo import summary

In [21]:
model = resnet18()
model.eval()

# summary(model, input_size=(1, 3, 224, 224))

# 打印模型的输入输出


# 使用trace实现ir需要一个dummy_input
dummy_input = torch.randn(1, 3, 224, 224)

# ir
with torch.no_grad():
    jit_model = torch.jit.trace(model, dummy_input)

In [23]:
jit_layer1 = jit_model.layer1
print(jit_layer1.graph)

graph(%self.11 : __torch__.torch.nn.modules.container.___torch_mangle_415.Sequential,
      %4 : Float(1, 64, 56, 56, strides=[200704, 3136, 56, 1], requires_grad=0, device=cpu)):
  %_1.1 : __torch__.torchvision.models.resnet.___torch_mangle_414.BasicBlock = prim::GetAttr[name="1"](%self.11)
  %_0.1 : __torch__.torchvision.models.resnet.___torch_mangle_408.BasicBlock = prim::GetAttr[name="0"](%self.11)
  %6 : Tensor = prim::CallMethod[name="forward"](%_0.1, %4)
  %7 : Tensor = prim::CallMethod[name="forward"](%_1.1, %6)
  return (%7)



In [24]:
print(jit_layer1.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _1 = getattr(self, "1")
  _0 = getattr(self, "0")
  _2 = (_1).forward((_0).forward(argument_1, ), )
  return _2



In [25]:
# 展开计算图
torch._C._jit_pass_inline(jit_layer1.graph)
print(jit_layer1.graph)

graph(%self.11 : __torch__.torch.nn.modules.container.___torch_mangle_415.Sequential,
      %4 : Float(1, 64, 56, 56, strides=[200704, 3136, 56, 1], requires_grad=0, device=cpu)):
  %_1.1 : __torch__.torchvision.models.resnet.___torch_mangle_414.BasicBlock = prim::GetAttr[name="1"](%self.11)
  %_0.1 : __torch__.torchvision.models.resnet.___torch_mangle_408.BasicBlock = prim::GetAttr[name="0"](%self.11)
  %8 : float = prim::Constant[value=0.10000000000000001](), scope: __module.layer1/__module.layer1.0/__module.layer1.0.bn1 # /home/melon/miniconda3/envs/torch240/lib/python3.12/site-packages/torch/nn/functional.py:2512:0
  %9 : float = prim::Constant[value=1.0000000000000001e-05](), scope: __module.layer1/__module.layer1.0/__module.layer1.0.bn1 # /home/melon/miniconda3/envs/torch240/lib/python3.12/site-packages/torch/nn/functional.py:2512:0
  %10 : NoneType = prim::Constant(), scope: __module.layer1/__module.layer1.0/__module.layer1.0.conv1
  %11 : int = prim::Constant[value=1](), scope:

In [26]:
# 序列化
jit_model.save("resnet18.pth")
jie_model = torch.jit.load("resnet18.pth")